In [66]:
import pandas as pd
import numpy as np

In [67]:
import os

print(os.getcwd())

/Users/buldiga/Documents/ml/book-recsys/notebooks


In [77]:
df = pd.read_csv('../large_files/interactions.csv')
users = pd.read_csv('../large_files/users.csv')

In [69]:
df["start_date"] = pd.to_datetime(df["start_date"])
min_date = df["start_date"].max().normalize() - pd.DateOffset(days=30)

In [64]:
merged = df.merge(users, on='user_id')

In [70]:
df[df['progress'] < 50].shape, df[df['progress'] > 50].shape

((826154, 5), (727426, 5))

In [71]:
submission = pd.read_csv("../large_files/sample_submission.csv")
pred_pop = pd.DataFrame({"user_id": submission["Id"].unique()})

In [72]:
user_ids_pred = set(list(pred_pop['user_id']))

In [73]:
user_ids = set(list(users['user_id']))

In [74]:
len(user_ids_pred.difference(user_ids)), len(user_ids_pred.intersection(user_ids))

(547, 2527)

In [14]:
usr = pred_pop.merge(users, how='left', on='user_id').fillna(1.0)

In [47]:
groups = merged.loc[merged["start_date"] > min_date, ["item_id", 'age']].fillna('25_34').groupby('age')

In [59]:
rec = {}
for name, group in groups:
    rec[name] = group['item_id'].value_counts().head(10).index.values
print(rec)

{'18_24': array([ 80003, 283713, 184549, 385281, 287060, 267817, 276903, 168963,
       112869,  56877]), '25_34': array([ 80003, 357309, 385281, 184549, 276903,  56877, 283713, 287060,
       168963, 262464]), '35_44': array([184549, 283713, 357309, 385281, 168963,  80003, 242176, 276903,
       287060,  56877]), '45_54': array([283713, 184549, 143175, 168963, 276903,  55466, 357309, 242176,
       321351,  50718]), '55_64': array([283713, 184549, 143175,  55466, 168963, 276903, 160349,  51423,
       323949,  79499]), '65_inf': array([283713, 184549, 143175,  55466, 160349,  51423, 374648, 276903,
       178529, 168963])}


In [90]:
counts = users.groupby('age').count()['user_id']

In [93]:
normalized_df=counts/counts.sum()


In [94]:
normalized_df

age
18_24     0.207025
25_34     0.267414
35_44     0.228593
45_54     0.164933
55_64     0.086000
65_inf    0.046035
Name: user_id, dtype: float64

In [97]:
missing = merged['age'].isnull()


In [100]:
normalized_df.values

array([0.20702544, 0.26741424, 0.22859287, 0.16493253, 0.08600012,
       0.0460348 ])

In [101]:
merged.loc[missing,'age'] = np.random.choice(normalized_df.index, size=len(merged[missing]),p=normalized_df.values)
